In [ ]:
import pandas as pd
import numpy as np
import scipy
import pysr
import sympy
import math
from pysr import PySRRegressor
import matplotlib.pyplot as plt
import pickle

In [ ]:
#This doesn't work to install torch, so I likely need to run this in the command line, though I will leave it here for reference
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu

In [ ]:
import torch
import torchvision
from torchvision import datasets
from torchvision.transforms import ToTensor

In [ ]:
pip install sxs

In [ ]:
import sxs
print(sxs.__file__)
print(sxs.__version__)
print(dir(sxs))

In [ ]:
def dataframe(non_eccentric, non_spinning, aligned_spin, not_deprecated):
    df = sxs.load("dataframe")
    df = df.loc[np.isfinite(df["common_horizon_time"])]
    if non_eccentric:
        df = df.loc[df['reference_eccentricity'] < 0.01]
    if non_spinning:
        df = df.loc[df["reference_dimensionless_spin1_mag"] < 0.001]
        df = df.loc[df["reference_dimensionless_spin2_mag"] < 0.001]
    if aligned_spin:
        df = df.loc[df["reference_dimensionless_spin1_x"] < 0.001]
        df = df.loc[df["reference_dimensionless_spin2_x"] < 0.001]
        df = df.loc[df["reference_dimensionless_spin1_y"] < 0.001]
        df = df.loc[df["reference_dimensionless_spin2_y"] < 0.001]
    if not_deprecated:
        df = df.loc[df["deprecated"] == False]
        df = df.drop('SXS:BBH:0621') #All of my best models were having a hard time fitting CHT for this simulation. Seems like an outlier
    return df    
    
df = dataframe(non_eccentric = True, non_spinning = False, aligned_spin = False, not_deprecated = True)
print(len(df))

In [ ]:

#Here I write some code to convert the simulation parameters that Vaishak gave me into the parameters I need to predict CHT from my models
m_1 = 0.554763952862425
m_2 = 0.445251975007412
q = m_1/m_2
q_n = q/np.max(df["reference_mass_ratio"])

s_1 =  np.array([-0.014131739872718789, 0.17993900077887057, -0.39832502308873785])
s_2 =  np.array([-0.4676236498668915, -0.008958494480525601, 0.5435607064742118])

L = np.array([0.0008175197573002864, 0.0009421312931779968, 0.015290980276651522])
L_mag = np.sqrt(np.dot(L, L))
T = 2*np.pi/L_mag
T_n = T/np.max(ref_orb_period)

def chi_eff(m_1, m_2, s_1, s_2, L):
    return (m_1*np.dot(s_1, L) + m_2*np.dot(s_2, L))/(m_1 + m_2)

def chi_perp(s, L):
    x = np.cross(s, L)
    return np.sqrt(np.dot(x, x))

X_eff = chi_eff(m_1, m_2, s_1, s_2, L)

X1_perp = chi_perp(s_1, L)

X2_perp = chi_perp(s_2, L)

sim = np.array([[T, q, X_eff, X1_perp, X2_perp]])
print(sim)

In [ ]:

#Here I write some code to convert the simulation parameters that Vaishak gave me into the parameters I need to predict CHT from my models
m_1 = 0.554763952862425
m_2 = 0.445251975007412
q = m_1/m_2
q_n = q/np.max(df["reference_mass_ratio"])

s_1 =  np.array([-0.014131739872718789, 0.17993900077887057, -0.39832502308873785])
s_2 =  np.array([-0.4676236498668915, -0.008958494480525601, 0.5435607064742118])

L = np.array([0.0008175197573002864, 0.0009421312931779968, 0.015290980276651522])
L_mag = np.sqrt(np.dot(L, L))
T = 2*np.pi/L_mag
T_n = T/np.max(ref_orb_period)

def chi_eff(m_1, m_2, s_1, s_2, L):
    return (m_1*np.dot(s_1, L) + m_2*np.dot(s_2, L))/(L_mag*(m_1 + m_2))

def chi_perp(s, L):
    x = np.cross(s, L)
    return np.sqrt(np.dot(x, x))

X_eff = chi_eff(m_1, m_2, s_1, s_2, L)

X1_perp = chi_perp(s_1, L)

X2_perp = chi_perp(s_2, L)

sim_nn = torch.tensor([T_n, q_n, X_eff, X1_perp, X2_perp])
sim_nn = sim_nn.float()
sim_sr = np.array([[T, q, X_eff, X1_perp, X2_perp]])

In [ ]:
#This first block subtracts corrections due to varying mass ratio from quadrupolar, Newtonian model
init_param_q = np.array([[T, q]])
with open("best_model_0.446.pk", 'rb') as file:
    loaded_model_q = pickle.load(file)

#This second block subtracts spin corrections from quadrupolar, Newtonian model
init_param_spin = np.array([[T, q, X_eff]])
with open("chi_spin_model_0.865.pk", 'rb') as file:
    loaded_model_spin = pickle.load(file)

#This third block subracts precession corrections from quadrupolar, Newtonian model
init_param_prec = np.array([[T, q, X1_perp, X2_perp]])
with open("chi_spin_model_0.749.pk", 'rb') as file:
    loaded_model_prec = pickle.load(file)
    
corrections = loaded_model_q.predict(init_param_q) + loaded_model_spin.predict(init_param_spin) + loaded_model_prec.predict(init_param_prec) #Compile corrections from mass ratio, spin, and precession

corrected_CHT = Newtonian_CHT([T, q]) - corrections #Subtracts "post-Newtonian" corrects from the Newtonian predictions for CHT

print(corrected_CHT)

In [ ]:
def weighted_E(outputs, labels):
   return (outputs - labels)/labels #For every simulation in a batch, I take the difference between the CHT proportion
    #associated with that simulation and the CHT proportion predicted by my model weighted by the simulation proportion. This should give the proportion
    #residual of my model's CHT predictions, then I sum up over the batch and divide by the size of the batch
    
criterion = weighted_E

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self, activation_function):
        super().__init__()
        self.fc1 = nn.Linear(5, 120)
        self.fc2 = nn.Linear(120, 120)
        self.fc3 = nn.Linear(120, 84)
        self.fc4 = nn.Linear(84, 1)
        self.activation_function = activation_function
    
    def forward(self, x):
        x = self.activation_function(self.fc1(x))
        x = self.activation_function(self.fc2(x))
        x = self.activation_function(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
net = Net(nn.ReLU())
model_path = 'nnet_33177.pth'

net.load_state_dict(torch.load(model_path, weights_only=False))
net.eval()

In [ ]:
CHT_pred = net(sim)
CHT_pred = float(CHT_pred)
print("Common horizon time is " + str(round(float(CHT_pred*np.max(df["common_horizon_time"] - df["reference_time"])), 1)))